### 수집된 포털 정보기반 딥크롤링
* 기존에 수집된 정보를 바탕으로 매체별 크롤링
* 다양한 서브 데이터의 정형화
* URL 기반의 다양한 크롤링 기법 사용

### 분석요소
* 제목 :
* 내용  :
* 작성일 :
* 매체: 
* 작성자:


### 기존 수집 데이터 활용


In [8]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [9]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [10]:
calcDate('2021.2.1')

'2021-2-1 00:00:00.000000'

In [11]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [5]:
def getData(whr=' where nstat=0'):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select nidx,ncomp,nurl from nnews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [12]:
def setStat(index=0,stat=0):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='update nnews set nstat={1} where nidx={0}'.format(index,stat)
        res=cursor.execute(sql)
        if (stat==1):
            print('-',end='')
        else:
            print('=',end='')

In [13]:
setStat(67,1)

OperationalError: no such column: nstat

In [65]:

def insSub(d):
    dbname='navernews.db'
    if(len(d)==7):
        d.append(makeKey(d[6]))
    else:
        print(d)
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from subnews where skey="'+d[7]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[0])
        else:
            sql='insert into subnews (stitle,swriter,sday,sarticle,smidx,scomp,surl,skey) values(?,?,?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
        

In [55]:
def getAsia(urls):
    setStat(urls[0],1)# 크롤링 시작 업데이트
    url=urls[2]
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    title_block=htmls.select('div.area_title')[0]
    wtitle=title_block.select('h3')[0].text.strip()
    writer=htmls.select('.article_view > .article .e_article')[0].text.strip()
    wday_=title_block.select('p.user_data')
    wday=str(wday_).split('</strong>')[-1].replace('</p>]','').replace('.','-').strip()
    #print(wday)
    article_=htmls.select('.article_view > .article ')[0]
    pattern='<!-- 본문// -->[\S\s]*'
    article__=re.sub(pattern,' ',str(article_))
    warticle=bsp(article__,'html.parser').text.strip()
    #print(warticle)
    """
    pattern='<script[^>]*>((\n|\r|.)*?)<\/script>'
    article__=re.sub(pattern,' ',str(article_))
    """
    return [wtitle,writer,wday,warticle,urls[0],urls[1],urls[2]]
    #return [wtitle,wday,urls[0],urls[1],urls[2]]
urls=(69, '아시아경제', 'https://view.asiae.co.kr/article/2021031715545233623')
#print(getAsia(urls))

In [56]:
def getDonga(urls):
    setStat(urls[0],1)# 크롤링 시작 업데이트
    url=urls[2]
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    title_block=htmls.select('div.article_title')[0]
    wtitle=title_block.select('h1.title')[0].text.strip()
    writer=title_block.select('span.report')[0].text.replace('동아닷컴 기자','').strip()
    wday=title_block.select('span.date01')[-1].text.strip()
    article_=htmls.select('div.article_txt')[0]
    pattern='<script[^>]*>((\n|\r|.)*?)<\/script>'
    article__=re.sub(pattern,' ',str(article_))
    pattern='<div id="bestnews_layer">.*'
    article__=re.sub(pattern,' ',str(article__))
    warticle=bsp(article__,'html.parser').text
    return [wtitle,writer,wday,warticle,urls[0],urls[1],urls[2]]

In [57]:
def getDona(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('div.article_title')
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div.news_wrap')
        subartis=tgt.select('ul.list_cluster > li.sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a.news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a.news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a.sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a.sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)



### 비동기 웹페이지 크롤링
* 크롬 드라이버 활용
* 다운로드: https://sites.google.com/a/chromium.org/chromedriver/downloads
* 해당 크롬 버전에 맞는 드라이버 다운로드
* pip install selenium


In [59]:
from selenium import webdriver
from bs4 import BeautifulSoup as bsp

In [60]:
dpath='D:\ASIA\chromedriver'
driver= webdriver.Chrome(dpath)

In [61]:
url='https://www.chosun.com/economy/market_trend/2021/03/17/7O5JTHZSJVAJBFMUWNJGK22XUA/?utm_source=naver&utm_medium=referral&utm_campaign=naver-news'
urls=[327,'조선일보',url]

In [62]:
def getChosun(urls):
    setStat(urls[0],1)# 크롤링 시작 업데이트
    url=urls[2]
    driver.get(url)
    driver.implicitly_wait(3)
    html=driver.page_source
    src=bsp(html,'html.parser')
    wtitle=src.select('h1.article-header__headline')[0].text
    writer=src.select('a.article-byline__author')[0].text.replace('기자','').strip()
    wdate_=src.select('div.article-dateline')[0].text.split("|")[-1]
    wday=wdate_.replace('수정','').replace('.','-').strip()+':00'
    warticle=src.select('section.article-body')[0].text.replace('\xad','')
    #urls: main index,company,url

In [63]:
res=getChosun(urls)
print(res)

-None


In [66]:
allurl=getData()
allurl[:5]
for urls in allurl[:3]:
    print('-'*50)
    wdata=''
    #print(nidx,ncomp,turl)
    if urls[1]=="동아일보":
        wdata=getDonga(urls)
        insSub(wdata)
        setStat(urls[0],2)# 크롤링 시작 업데이트
    elif urls[1]=="아시아경제":
        wdata=getAsia(urls)
        insSub(wdata)
        setStat(urls[0],2)# 크롤링 시작 업데이트
    elif urls[1]=="조선일보":
        wdata=getChosun(urls)
        insSub(wdata)
        setStat(urls[0],2)# 크롤링 시작 업데이트
    print(wdata)
    

--------------------------------------------------
-.=['신세계 정용진 지난해 연봉 33억…코로나19 영향으로 일부 감소', '김혜린', '수정 2021-03-17 14:45', '\n신세계그룹 제공 정용진 신세계그룹 부회장이 지난해 연봉으로 33억 6800만 원을 받았다. 신세계 그룹 총수 일가는 총 142억 원을 수령했다.17일 신세계·이마트 사업보고서에 따르면 정 부회장은 지난해 이마트에서 총 33억 6800만 원을 받았다. 급여 20억 3400만 원과 상여 13억 3400만 원이다.이마트 측은 “어려운 대내외 경영 환경 속에서도 매출액 14조 2138억 원과 영업이익 2950억 원을 달성한 점을 고려했다”고 밝혔다.\n\n동생 정유경 신세계 총괄사장은 급여와 상여금 각각 17억 9400만 원, 11억 6600만 원으로 총 29억 6000만 원을 수령했다.\n주요기사\n\n산사태까지 난 땅…용인시 공무원 매입 11일뒤 ‘특구’ 발표원로가 전한 시대정신과 인재론‘윤석열의 정치’ 보인다\n\n신세계 측은 “기존 백화점 경쟁력 강화로 어려운 대내외 환경 속에서도 영업이익 1268억 원을 달성한 점을 고려했다”고 전했다. 정 부회장과 정 총괄사장 모두 연봉이 전년 대비 감소한 것으로 나타났다. 정 부회장은 약 1억 9400만 원, 정 총괄사장은 1억 5400만 원가량을 덜 받았다. 이는 신종 코로나바이러스 감염증(코로나19) 사태로 실적이 감소한 영향으로 보인다.이명희 신세계그룹 회장은 이마트에서 26억 9300만 원, 신세계에서 12억 6100만 원으로 총 39억 5400만 원을 받았다. 정재은 명예회장도 이와 동일한 연봉을 수령했다.김혜린 동아닷컴 기자 sinnala8@donga.com\n', 67, '동아일보', 'https://www.donga.com/news/article/all/20210317/105923783/2', 'wwwdongacomnewsarticleall202103171059237832']
----